In [1]:
from functools import partial
from pathlib import Path
import time
from zipfile import ZipFile

import napari
from napari.utils import nbscreenshot

from shapely.geometry import GeometryCollection
from shapely.geometry import Point

from scipy.ndimage import affine_transform
from skimage.transform import AffineTransform

import dask.array as da
from dask.array.core import normalize_chunks
#from dask_image.ndfilters import gaussian_filter

#from spacem_lib.io.writer import write_ome_zarr

from utils.download_sample import download_from_dropbox
from utils.coordinate_utils import extract_coordinates, normalize_coords_to_pixel
from utils.imutils import *
from utils.shapely_and_napari_utils import *
from utils.chunks import get_chunk_coordinates, get_rect_from_chunk_slice, tile_chunk_intersections
from fuse.fuse import fuse_func

In [2]:
%gui qt

In [3]:
time.sleep(1)

In [4]:
nikonfolder = "/g/scb/alexandr/shared/Luisa/HT_test/20210621/PostMaldi_Microscopy/Whole_Slide/"

In [5]:
coords = extract_coordinates(Path(nikonfolder)/"out.txt")
coords

,name,X,Y,Z,um/px
0,#1,7471.8,9216.6,2315.75,0.64
1,#2,6417.3,9222.6,2315.75,0.64
2,#3,5362.8,9228.6,2315.75,0.64
3,#4,4308.3,9234.6,2315.75,0.64
4,#5,3253.8,9240.6,2315.72,0.64
...,...,...,...,...,...
667,#668,3163.2,-6700.8,2315.70,0.64
668,#669,4217.7,-6706.8,2315.70,0.64
669,#670,5272.2,-6712.8,2315.70,0.64
670,#671,6326.7,-6718.8,2315.70,0.64


In [6]:
files = sorted(list(Path(nikonfolder).glob("*.tif")))

In [7]:
#def subsample(arr, factor: int=2):
#    return arr[::factor, ::factor]

In [8]:
normalized_coords=normalize_coords_to_pixel(coords).to_numpy()

In [9]:
limit_images = 90
if limit_images:
    files=files[:limit_images]
    normalized_coords=normalized_coords[:limit_images]

In [10]:
factor = 16
_subsample = partial(subsample, factor=factor)

load_transforms = [crop_black_border,
                    _subsample,]
                   # transpose]
_load_image = partial(load_image, transforms=load_transforms)

In [11]:
images = list(map(_load_image, files))

In [12]:
eight_transforms = ((), 
                    (transpose,), 
                    (np.fliplr,), 
                    (np.flipud,), 
                    (np.flipud, transpose,), 
                    (np.fliplr, transpose,), 
                    (np.flipud, np.fliplr,), 
                    (np.flipud, np.fliplr, transpose,),)

In [13]:
chunk_size = (4096,4096)
_fuse_func=partial(fuse_func, 
                   imload_func=None,
                   dtype=np.uint8)
previews =[]
for transform_chain in eight_transforms:
    mosaics_shape_images = get_mosaics_bboxes_transforms(images,transform_chain,normalized_coords,factor)
    _tmp = get_chunk_slices_and_shapes(chunk_size=chunk_size, array_shape=mosaics_shape_images['stitched_shape'])
    chunk_slices = _tmp['chunk_slices']
    chunks_shapely = _tmp['chunks_shapely']
    chunk_tiles = tile_chunk_intersections(mosaics_shape_images['mosaic_shifted'],
                                       mosaics_shape_images['transformed_images'],
                                       mosaics_shape_images['transforms'],
                                       chunks_shapely, 
                                       chunk_slices)
    target_array = da.map_blocks(func=_fuse_func,
                             chunks=normalize_chunks(chunk_size, mosaics_shape_images['stitched_shape']), 
                             input_tile_info=chunk_tiles,
                             dtype=np.uint8)
    previews.append(target_array.compute())

Processing chunk at (0, 0)
Processing chunk at (4096, 0)
Processing chunk at (0, 0)
Processing chunk at (4096, 0)
Processing chunk at (0, 0)Processing chunk at (4096, 0)

Processing chunk at (0, 0)
Processing chunk at (4096, 0)
Processing chunk at (0, 0)Processing chunk at (4096, 0)

Processing chunk at (0, 0)Processing chunk at (4096, 0)

Processing chunk at (0, 0)Processing chunk at (4096, 0)

Processing chunk at (0, 0)Processing chunk at (4096, 0)



In [14]:
v = napari.Viewer()

In [15]:
[v.add_image(p) for p in previews]

[<Image layer 'p' at 0x7f7ed40237c0>,
 <Image layer 'p [1]' at 0x7f7f10343370>,
 <Image layer 'p [2]' at 0x7f7eb80803a0>,
 <Image layer 'p [3]' at 0x7f7eb8093ac0>,
 <Image layer 'p [4]' at 0x7f7eb0379970>,
 <Image layer 'p [5]' at 0x7f7eb02f7a30>,
 <Image layer 'p [6]' at 0x7f7eb02768e0>,
 <Image layer 'p [7]' at 0x7f7eb01f5e80>]